<a href="https://colab.research.google.com/github/michel-wue/Seminar/blob/Tries_API_Data_extraction/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests as r
import pandas as pd
from pandas import json_normalize
import re
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
Data_basic = pd.DataFrame()

In [20]:
Data_basic = pd.DataFrame(columns=['GameID','season','types','week','HomeTeam_ID','AwayTeam_ID','Winner'])
week = list(range(1, 19))
seasons=list(range(2021,2023))
time=list(range(0,901,60))

In [ ]:
time

[60, 120, 180, 240, 300, 360, 420, 480, 540, 600, 660, 720, 780, 840, 900]

In [21]:

def infos(season,types,weeks):
  url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{season}/types/{types}/weeks/{weeks}/events"
  response = r.get(url)
  json_data = response.json()
  df = json_normalize(json_data['items'])
  for game in range(len(df)):
    l=[]
    response = r.get(df['$ref'][game])
    data=response.json()
    try:
      df2=json_normalize(data['competitions'])
      l.append(df2['id'][0])
      l.append(season)
      l.append(types)
      l.append(weeks)
      l.append(df2['competitors'][0][0]['id'])
      l.append(df2['competitors'][0][1]['id'])
      try:
        if df2['competitors'][0][0]['winner'] == True:
          l.append('Home')
        else:
          l.append('Away')
      except KeyError:
        l.append('NaN')
    except KeyError:
      print('it doesnt worked')
    Data_basic.loc[len(Data_basic)] = l


In [22]:
for x in seasons:  
  for i in week:
    infos(x,2,i)

In [ ]:
#Data_basic.to_csv('/content/drive/MyDrive/play_ID_data.csv', index=False)


In [31]:
Data_basic

,GameID,season,types,week,HomeTeam_ID,AwayTeam_ID,Winner
0,401326322,2021,2,1,27,6,Home
1,401326315,2021,2,1,1,21,Away
2,401326308,2021,2,1,2,23,Away
3,401326316,2021,2,1,4,16,Home
4,401326317,2021,2,1,8,25,Away
...,...,...,...,...,...,...,...
539,401437957,2022,2,18,21,19,Home
540,401437962,2022,2,18,25,22,Home
541,401437963,2022,2,18,26,14,Home
542,401437959,2022,2,18,28,6,Home


In [23]:
Data_basic

,GameID,season,types,week,HomeTeam_ID,AwayTeam_ID,Winner
0,401326322,2021,2,1,27,6,Home
1,401326315,2021,2,1,1,21,Away
2,401326308,2021,2,1,2,23,Away
3,401326316,2021,2,1,4,16,Home
4,401326317,2021,2,1,8,25,Away
...,...,...,...,...,...,...,...
539,401437957,2022,2,18,21,19,Home
540,401437962,2022,2,18,25,22,Home
541,401437963,2022,2,18,26,14,Home
542,401437959,2022,2,18,28,6,Home


In [26]:
stats_df = pd.DataFrame()
stats_df = pd.DataFrame(columns=['GameID','Pass_Incompletion_Home',
  'Punt_Home',
  'Timeout_Home',
  'Punt_Away',
  'Pass_Incompletion_Away',
  'Timeout_Away',
  'homeScore',
  'awayScore',
  'PassReception_Home_yards',
  'PassReception_Away_yards',
  'Penalty_Home',
  'Penalty_Away',
  'Penalty_Home_yards',
  'Penalty_Away_yards',
  'rushing_yards_home',
  'rushing_yards_away',
  'field_goals_good_home',
  'field_goals_good_away',
  'field_goals_missed_home',
  'field_goals_missed_away',
  'fumble_recovery_oponnent_home',
  'fumble_recovery_oponnent_away',
  'fumble_recovery_own_home',
  'fumble_recovery_own_away',
  'sack_yards_home',
  'sack_yards_away',
  'passing_TD_home',
  'passing_TD_away',
  'rushing_TD_home',
  'rushing_TD_away',
  'interception_home',
  'interception_away',
  'Interception_Return_TD_home',
  'Interception_Return_TD_away',
  'kick_off_return_TD_home',
  'kick_off_return_TD_away',
  'safety_home',
  'safety_away'])

In [27]:
def stats(GameID,Quarter):
  url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
  response_drive = r.get(url)
  drive_data = response_drive.json()
  drive_df = json_normalize(drive_data['items'])
  data=[]
  Pass_Incompletion_Home=0
  Punt_Home=0
  Timeout_Home=0
  Punt_Away=0
  Pass_Incompletion_Away=0
  Timeout_Away=0
  homeScore=0
  awayScore=0
  PassReception_Home_yards=0
  PassReception_Away_yards=0
  Penalty_Home=0
  Penalty_Away=0
  Penalty_Home_yards=0
  Penalty_Away_yards=0
  rushing_yards_home=0
  rushing_yards_away=0
  field_goals_good_home=0
  field_goals_good_away=0
  field_goals_missed_home=0
  field_goals_missed_away=0
  fumble_recovery_oponnent_home=0
  fumble_recovery_oponnent_away=0
  fumble_recovery_own_home=0
  fumble_recovery_own_away=0
  sack_yards_home=0
  sack_yards_away=0
  passing_TD_home=0
  passing_TD_away=0
  rushing_TD_home=0
  rushing_TD_away=0
  interception_home=0
  interception_away=0
  Interception_Return_TD_home=0
  Interception_Return_TD_away=0
  kick_off_return_TD_home=0
  kick_off_return_TD_away=0
  safety_home=0
  safety_away=0
  for index, row in drive_df.iterrows():
    try:
      if (re.search(r"teams/(\d+)",row['team.$ref']).group(1) == Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]) &(row['start.period.number']==Quarter):
        for detail in range(len(row['plays.items'])):
          if 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Incompletion':
            Pass_Incompletion_Home=Pass_Incompletion_Home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Punt':
            Punt_Home=Punt_Home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Timeout':
            Timeout_Home=Timeout_Home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Reception':
            PassReception_Home_yards=PassReception_Home_yards+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Penalty':
            Penalty_Home_yards=Penalty_Home_yards+row['plays.items'][detail]['statYardage']
            Penalty_Home=Penalty_Home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rush':
            rushing_yards_home=rushing_yards_home+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Sack':
            sack_yards_home=row['plays.items'][detail]['statYardage']+sack_yards_home
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Passing Touchdown':
            passing_TD_home=passing_TD_home+1
            PassReception_Home_yards=PassReception_Home_yards+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rushing Touchdown':
            rushing_TD_home=rushing_TD_home+1
            rushing_yards_home=rushing_yards_home+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Interception Return':
            interception_home=interception_home+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Good':
            field_goals_good_home=field_goals_good_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Missed':
            field_goals_missed_home=field_goals_missed_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Interception Return Touchdown':
            Interception_Return_TD_home=Interception_Return_TD_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Kickoff Return Touchdown':
            kick_off_return_TD_home=kick_off_return_TD_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Own)':
            fumble_recovery_own_home=fumble_recovery_own_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Opponent)':
            fumble_recovery_oponnent_home=fumble_recovery_oponnent_home+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Safety':
            safety_home=safety_home+1
          homeScore=row['plays.items'][detail]['homeScore']
      elif (re.search(r"teams/(\d+)",row['team.$ref']).group(1) != Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]) &(row['start.period.number']==Quarter):
        for detail in range(len(row['plays.items'])):
          if 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Incompletion':
            Pass_Incompletion_Away=Pass_Incompletion_Away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Punt':
            Punt_Away=Punt_Away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Timeout':
            Timeout_Away=Timeout_Away+1  
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Reception':
            PassReception_Away_yards=PassReception_Away_yards+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Penalty':
            Penalty_Away_yards=Penalty_Away_yards+row['plays.items'][detail]['statYardage']
            Penalty_Away=Penalty_Away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rush':
            rushing_yards_away=rushing_yards_away+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Sack':
            #print(row['plays.items'][detail]['id'])
            #print(row['plays.items'][detail]['statYardage'])
            sack_yards_away=row['plays.items'][detail]['statYardage']+sack_yards_away
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Passing Touchdown':
            passing_TD_away=passing_TD_away+1
            PassReception_Away_yards=PassReception_Away_yards+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Rushing Touchdown':
            rushing_TD_away=rushing_TD_away+1
            rushing_yards_away=rushing_yards_away+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Pass Interception Return':
            interception_away=interception_away+row['plays.items'][detail]['statYardage']
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Good':
            field_goals_good_away=field_goals_good_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Field Goal Missed':
            field_goals_missed_away=field_goals_missed_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Interception Return Touchdown':
            Interception_Return_TD_away=Interception_Return_TD_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Kickoff Return Touchdown':
            kick_off_return_TD_away=kick_off_return_TD_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Own)':
            fumble_recovery_own_away=fumble_recovery_own_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Fumble Recovery (Opponent)':
            fumble_recovery_oponnent_away=fumble_recovery_oponnent_away+1
          elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type']['text'] == 'Safety':
            safety_away=safety_away+1
          awayScore=row['plays.items'][detail]['awayScore']
    except TypeError:
      print('it doenst worked')
  data = [    GameID,    Pass_Incompletion_Home,
          Punt_Home,    Timeout_Home,
          Punt_Away,    Pass_Incompletion_Away,
          Timeout_Away,    homeScore,
          awayScore,    PassReception_Home_yards,
          PassReception_Away_yards,    Penalty_Home,
          Penalty_Away,    Penalty_Home_yards,
          Penalty_Away_yards,    rushing_yards_home,
          rushing_yards_away,    field_goals_good_home,
          field_goals_good_away,    field_goals_missed_home,
          field_goals_missed_away,    fumble_recovery_oponnent_home,
          fumble_recovery_oponnent_away,    fumble_recovery_own_home,
          fumble_recovery_own_away,    sack_yards_home,
          sack_yards_away,    passing_TD_home,
          passing_TD_away,    rushing_TD_home,
          rushing_TD_away,    interception_home,
          interception_away,    Interception_Return_TD_home,
          Interception_Return_TD_away,    kick_off_return_TD_home,
          kick_off_return_TD_away,
          safety_home,    safety_away]
  stats_df.loc[len(stats_df)] = data







In [ ]:
#GameID='320909012'
#url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
#response_drive = r.get(url)
#drive_data = response_drive.json()
#drive_df = json_normalize(drive_data['items'])
#homescore=0
#awayScore=0
#Pass_Incompletion_Home=0
#Punt_Home=0
#Time_Out=0

#awayScore=0
#for plays in range(len(drive_df['plays.items'])):
  #for detail in range(len(drive_df['plays.items'][plays])):
    #if drive_df['plays.items'][plays][detail]['type']['text'] == 'Pass Incompletion':
      #Pass_Incompletion_Home=Pass_Incompletion_Home+1
    #elif drive_df['plays.items'][plays][detail]['type']['text'] == 'Punt':
      #Punt_Home=Punt_Home+1
    #elif drive_df['plays.items'][plays][detail]['type']['text'] == 'Timeout':
      #Time_Out=Time_Out+1
    #print(drive_df['plays.items'][plays][detail]['type']['text'])
    #print(drive_df['plays.items'][plays][detail]['statYardage'])
    #print(drive_df['plays.items'][plays][detail]['scoreValue'])
    #print(drive_df['plays.items'][plays][detail]['awayScore'])
    #print(drive_df['plays.items'][plays][detail]['homeScore'])
#print(Time_Out)

In [28]:
for plays in Data_basic['GameID']:
  stats(plays,1)
  

it doenst worked
it doenst worked


In [ ]:
#stats_df.to_csv('/content/drive/MyDrive/quarter1_data.csv', index=False)

In [29]:
stats_df

,GameID,Pass_Incompletion_Home,Punt_Home,Timeout_Home,Punt_Away,Pass_Incompletion_Away,Timeout_Away,homeScore,awayScore,PassReception_Home_yards,...,rushing_TD_home,rushing_TD_away,interception_home,interception_away,Interception_Return_TD_home,Interception_Return_TD_away,kick_off_return_TD_home,kick_off_return_TD_away,safety_home,safety_away
0,401326322,4,3,0,1,3,0,7,7,113,...,0,0,0,0,0,0,0,0,0,0
1,401326315,7,0,0,0,0,0,6,7,58,...,0,0,0,0,0,0,0,0,0,0
2,401326308,6,2,0,3,3,0,3,0,32,...,0,0,0,0,0,0,0,0,0,0
3,401326316,2,3,0,3,2,1,0,0,15,...,0,0,0,0,0,0,0,0,0,0
4,401326317,5,0,0,0,0,0,7,7,61,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,401437957,3,0,0,2,4,0,10,0,108,...,1,0,0,0,0,0,0,0,0,0
540,401437962,4,3,0,2,2,0,7,6,73,...,0,0,0,0,0,0,0,0,0,0
541,401437963,1,0,0,1,1,0,6,6,20,...,0,0,28,0,0,0,0,0,0,0
542,401437959,3,1,0,2,2,0,7,0,25,...,0,0,0,0,0,0,0,0,0,0


In [ ]:

url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/401326316/competitions/401326316/drives?lang=en&region=us'
response_drive = r.get(url)
drive_data = response_drive.json()
drive_df = json_normalize(drive_data['items'])
 



In [ ]:
import datetime

duration_str = "15:00"
duration_arr = duration_str.split(":")
duration = datetime.timedelta(minutes=int(duration_arr[0]), seconds=int(duration_arr[1]))

print(duration)

0:15:00


In [ ]:
for index, row in drive_df.iterrows():
  for detail in range(len(row['plays.items'])):
    #print(row['plays.items'][detail])

    duration_str = row['plays.items'][detail]['clock']['displayValue']
    duration_arr = duration_str.split(":")
    duration = datetime.timedelta(minutes=int(duration_arr[0]), seconds=int(duration_arr[1]))
    print(duration=datetime.timedelta(seconds=900)-duration)

0:00:00
0:00:00
0:00:00
0:00:35
0:00:50
0:01:27
0:01:54
0:02:30
0:02:44
0:03:11
0:03:16
0:03:57
0:04:07
0:04:44
0:05:14
0:05:54
0:05:57
0:06:32
0:06:39
0:07:13
0:07:46
0:07:46
0:07:50
0:08:00
0:08:35
0:09:16
0:09:52
0:09:59
0:10:29
0:11:12
0:11:55
0:12:37
0:13:23
0:13:29
0:14:11
0:14:55
0:15:00
0:15:00
0:15:00
0:00:00
0:00:04
0:00:27
0:01:05
0:01:47
0:02:26
0:03:10
0:03:46
0:04:24
0:04:51
0:05:26
0:05:52
0:06:01
0:06:38
0:07:15
0:07:52
0:07:52
0:08:21
0:08:21
0:08:21
0:08:55
0:09:36
0:10:17
0:10:58
0:11:44
0:12:20
0:12:58
0:13:04
0:13:04
0:13:13
0:13:13
0:13:13
0:13:16
0:13:21
0:13:28
0:13:38
0:13:37
0:13:49
0:14:07
0:14:13
0:14:25
0:14:25
0:14:25
0:14:25
0:14:30
0:14:30
0:14:36
0:14:40
0:14:40
0:14:43
0:14:51
0:15:00
0:00:00
0:00:00
0:00:39
0:01:20
0:01:57
0:02:33
0:03:12
0:03:17
0:04:01
0:04:38
0:05:16
0:05:45
0:05:45
0:05:49
0:06:20
0:06:56
0:07:36
0:08:11
0:08:35
0:08:39
0:09:19
0:09:29
0:10:12
0:10:55
0:11:44
0:11:46
0:11:50
0:12:26
0:12:36
0:12:36
0:12:36
0:13:15
0:13:56
0:14:31


In [ ]:
print(duration-datetime.timedelta(seconds=110))

0:00:10


900